In [23]:
import math
from scipy.stats import norm

# Parameters
T = 2.5       
t = 0.0       
X = 1500      
r = 0.0386    
q = 0.038     
sigma = 0.2227  

# Defining s-values from 1125 to 1875 in 11 steps
x =  [1125, 1200, 1275, 1350, 1425, 1500, 1575, 1650, 1725, 1800, 1875]

#Defining d1&d2 and final price
def d1(S):
    numerator_d1 = math.tan(S/X - 1) + r * (T - t) * math.log(sigma * math.sqrt(T - t))
    denominator = 1 - math.cos(math.sqrt(sigma) * (T-t)**0.25)

    d1_value = numerator_d1 / denominator
    return d1_value

def d2(S):
    numerator_d2 = math.tan(S/X - 1) - sigma * math.sqrt(T - t) * (math.exp((r - q)/sigma**2) - 1)
    denominator = 1 - math.cos(math.sqrt(sigma) * (T-t)**0.25)
    d2_value = numerator_d2 / denominator
    return d2_value

def Pi(S):
    # Compute d1 and d2
    d1_val = d1(S)
    d2_val = d2(S)
    
    # term1 = S e^{(r-q)(T-t)} N(d1)
    term1 = S * math.exp((q - r) * (T - t)) * norm.cdf(d1_val)
    
    # term2 = X (1 + X/S)^{3/2} e^{-q(T-t)} N(d2)
    term2 = X * (1.0 + X/S)**0.5 * math.exp(-q * (T - t)) * norm.cdf(d2_val)
    
    return term1 - term2

# Print header
print(f"{'S':>6} | {'d1(S)':>12} | {'d2(S)':>12} | {'Pi(S)':>12}")
print("-"*100)

# Loop over each S in x and calculate d1(S), d2(S), and Pi(S)
for S in x:
    d1_val = d1(S)
    d2_val = d2(S)
    pi_val = Pi(S)
    print(f"{S:>6.1f} | {d1_val:12.6f} | {d2_val:12.6f} | {pi_val:12.6f}")


     S |        d1(S) |        d2(S) |        Pi(S)
----------------------------------------------------------------------------------------------------
1125.0 |    -2.082825 |    -1.518700 |  -113.294319
1200.0 |    -1.774954 |    -1.210828 |  -185.693291
1275.0 |    -1.473265 |    -0.909140 |  -275.975899
1350.0 |    -1.176106 |    -0.611981 |  -374.211730
1425.0 |    -0.881916 |    -0.317790 |  -464.692612
1500.0 |    -0.589195 |    -0.025070 |  -529.071124
1575.0 |    -0.296475 |     0.267651 |  -551.090664
1650.0 |    -0.002284 |     0.561841 |  -521.331076
1725.0 |     0.294875 |     0.859000 |  -440.162819
1800.0 |     0.596563 |     1.160689 |  -317.658601
1875.0 |     0.904435 |     1.468561 |  -170.386863


In [15]:
import math
from math import tan, cos, sqrt, log, exp
from scipy.stats import norm

# Given parameters
T     = 2.5      # Maturity
t     = 0.0      # Valuation time
X     = 1500.0   # Strike
r     = 0.0386   # Risk-free rate
q     = 0.038    # Dividend yield (or convenience yield)
sigma = 0.2227   # Volatility

# List of S values
S_values = [1125, 1200, 1275, 1350, 1425, 1500, 
            1575, 1650, 1725, 1800, 1875]

# Precompute some repeating terms for convenience
tau = T - t  # time-to-maturity
# The denominator that appears in both d1 and d2
denominator = 1.0 - cos( (sigma*0.5) * (tau*0.25) )

def d1(S):
    """
    Computes d1(S, t=0) using formula (1):
    d1(S,0) = [ tan(S/X - 1) + r * tau * ln( sigma * sqrt(tau) ) ]
              / [ 1 - cos( sigma^(1/2) * tau^(1/4) ) ]
    """
    numerator = tan((S/X) - 1.0) \
                + r * tau * log(sigma * sqrt(tau))
    return numerator / denominator

def d2(S):
    """
    Computes d2(S, t=0) using formula (2):
    d2(S,0) = [ tan(S/X - 1) 
                - sigma * sqrt(tau) * ( exp( (r - q)/sigma^2 ) - 1 ) ]
              / [ 1 - cos( sigma^(1/2) * tau^(1/4) ) ]
    """
    bracket = math.exp((r - q) / (sigma**2)) - 1.0
    numerator = tan((S/X) - 1.0) - sigma * sqrt(tau) * bracket
    return numerator / denominator

def Pi(S):
    """
    Computes Pi(S, t=0) using formula (3):
    Pi(S,0) = S * e^{(q-r)*tau} * N(d1) 
              - X * [ 1 + (X/S)^{1/2} * e^{-q*tau} ] * N(d2)
    """
    e_qr = exp((q - r) * tau)
    e_neg_q = exp(-q * tau)
    d1_val = d1(S)
    d2_val = d2(S)
    term1 = S * e_qr * norm.cdf(d1_val)
    term2 = X * (1.0 + (X/S)**0.5 * e_neg_q) * norm.cdf(d2_val)
    return term1 - term2

# Print header
print(f"{'S':>8} {'d1':>12} {'d2':>12} {'Pi(S,0)':>12}")
print("-"*44)

# Loop over S-values and compute results
for S in S_values:
    d1_val = d1(S)
    d2_val = d2(S)
    pi_val = Pi(S)
    print(f"{S:8.1f} {d1_val:12.6f} {d2_val:12.6f} {pi_val:12.6f}")

       S           d1           d2      Pi(S,0)
--------------------------------------------
  1125.0  -147.094580  -107.254581     0.000000
  1200.0  -125.351866   -85.511868     0.000000
  1275.0  -104.045837   -64.205839     0.000000
  1350.0   -83.059667   -43.219669     0.000000
  1425.0   -62.283184   -22.443186    -0.000000
  1500.0   -41.610497    -1.770498  -109.756712
  1575.0   -20.937809    18.902189 -2831.185670
  1650.0    -0.161326    39.678672 -2082.392453
  1725.0    20.824844    60.664842 -1049.577511
  1800.0    42.130873    81.970871  -947.908149
  1875.0    63.873586   103.713585  -847.862211
